In [466]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)
$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [587]:
def householder(vec):
    N = len(vec) #длина вектора х
    y = np.zeros(N) 
    y[0] = np.linalg.norm(vec)
    u1 = (vec - y ) / np.linalg.norm(vec - y) # просто следуем формуле, только у меня мой вектор u - это u.T из теории
    #так как я изначально вектор х задаю как строчку, а не как столбец
    u = np.zeros((1,N)) #создаем наш вектор, который сможем потом транспонировать
    u[0] = u1
    h = np.eye(N) - 2 * u.transpose() @ u
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    return vec@h, h

Test your function using tests below:

In [602]:
# Test I.1 (10% of the total grade).
#Раз проверка
v = np.array([1, 2, 3])
v1, h = householder(v)
assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1)

Удачно

In [603]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

Удачно

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [590]:
def householder1(vec): #для удобства запишу функцию похожую на первую, только она будет выводить просто матрицу H
    N = len(vec)
    y = np.zeros(N)
    y[0] = np.linalg.norm(vec)
    u1 = (vec - y ) / np.linalg.norm(vec - y)
    u = np.zeros((1,N))
    u[0] = u1
    h = np.eye(N) - 2 * u.transpose() @ u
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    return h

In [591]:
def qr1(a): # qr1 - функция, которая будет по заданной матрице выбирать первый столбец и по нему строить матрицу H
    # чтобы могли первый столбец занулить
    vec = a.transpose()[0]
    hh = householder1(vec)
    return hh

In [604]:
def qr_decomp(a): #приступим к написанию QR разложения
    N_0 = a.shape[0] #задаем необходимые константы N_0 - кол-во строк
    N_1 = a.shape[1] # N_1 - кол-во столбцов
    matrix_1 = np.zeros((N_0,N_0), dtype = float) #Из этой матрицы будем делать преобразование Householder'a
    matrix_2 = np.eye(N_0) # Эта матрица для того, чтобы начать создание матрицы Q, эту матрицу будем каждый раз
                            # Умножать на Householder и вследствии получим Q^(-1)
    matrix_a = a.copy() # Эта матрица для запуска цикла, то есть чтобы задать наше поочередное умножение, и потом
                        # Это перемножение даст нашу матрицу R 
    for i in range(0,N_1-1): #по условию преобразования производятся N-1 раз
        h1 = qr1(matrix_a[i:,i:]) #создаем матрицу Householder'a = H для обрезанной матрицы, так как на i - ом шаге
                                  #мы будем брать не полную матрицу
        h0 = np.eye(i) #это единичная матрица, которая будет стоять первой в конечной матрице H, она нужна
                       #чтобы мы оставшаяся матрица после вырезания при преобразовании не менялась
        matrix_1[:i,:i] = h0
        matrix_1[i:,i:] = h1 
        # тут идет как раз содание H, то есть сначала идет блок единичной матрицы, а потом блок для обрезанной матрицы
        matrix_a = matrix_1 @ matrix_a # идет последовательное применение H
        matrix_2 = matrix_1 @ matrix_2 # идет перемножение H, то есть создание Q^(-1)
        matrix_1 = np.zeros((N_0,N_0), dtype = float) #тут я не нашел другого выхода, кроме того как
                                                      #обнулить мою матрицу H, и в новом цикле просто по-новой ее 
                                                      #создать, вроде, это не должно быть чем-то страшным 
    r = matrix_a #просто вводим удобные обозначения
    q = np.linalg.inv(matrix_2) #находим обратную матрицу из произведения всех H
    return q , r
a1 = np.array(a, copy=True, dtype=float)
m, n = a1.shape

In [605]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [609]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Удачно

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [623]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)
assert_allclose(np.dot(qq, rr), qr_decomp(a)[0] @ qr_decomp(a)[1])

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

*Enter your explanation here* (10% of the total grade, peer-graded)

Результаты перемножения совпадают.НО! Если сравнивать по отдельности Q и R, то они не будут совпадать

In [624]:
rr

array([[-1.40152769, -1.2514379 , -0.89523615],
       [ 0.        ,  0.84158252,  0.68447942],
       [ 0.        ,  0.        , -0.5496046 ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

In [625]:
qr_decomp(a)[1]

array([[ 1.40152769,  1.2514379 ,  0.89523615],
       [ 0.        ,  0.84158252,  0.68447942],
       [ 0.        ,  0.        , -0.13872729],
       [ 0.        , -0.        ,  0.31721312],
       [ 0.        ,  0.        , -0.42684399]])

Можно заметить, что в библиотечном разложении, видимо, заложен какой-то другой алгоритм, потому что матрица R выглядит по-другому, точнее идея, кажется,немного другая. Потому что в нашем алгоритме сказано, что преоразование надо производить N-1 раз. Как минимум, в этом случае у нас N = 3. То есть у нас будет произведено только два преобразования, из-за чего поменяются у изначальной матрицы только первые два столбца( будет обнуление под диагоналями). А в библиотечной, видимо, преобразование применяется не N - 1 раз. По-другому я не могу обосновать, почему у нас ответы по отдельности не совпадают. ХОТЯ! Результат будет одинаковым( это дал нам понять тест, который мы провели выше)

assert_allclose(np.dot(qq, rr), qr_decomp(a)[0] @ qr_decomp(a)[1]) 

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [596]:
def qr11(a): #Создаем функцию, которая на входе будет получать матрицу и возвращать вектора u и матрицу R
    N0 = a.shape[0] #кол-во строк
    N1 = a.shape[1] #кол-во столбцов
    matrix = a.copy() # Также создаем матрицу, чтобы мы могли ее использовать в цикле
    matrix_u = np.zeros((N1-1,N0)) #создаем матрицу, которая будет хранить в себе вектора u
    for i in range(0, N1 - 1): #Преобразования производятся N - 1 раз
        b = matrix.copy() # Сделаем матрицу, из которой мы потом будем вынимать наши вектора
        x = b.transpose()[i] # выбираем i-ый вектор
        x[:i] = 0 # тут мы уже не берем вырезанную матрицу, а просто обнуляем все компоненты до i-ой компоненты
                  # и на i-ый компоненту ставим |x|
        y = np.zeros(N0)
        y[i] = np.linalg.norm(x) # вот тут мы вставляем норму х
        u = np.zeros((1,N0),dtype = float)
        u[0] = (x - y)/np.linalg.norm(x - y) # тут создаем вектор u, чтобы мы могли транспонировать его как раньше
        matrix_1 = u @ matrix #если я правильно понял, что от нас нужно, то именно это, что сначала я мою матрицу A
                              # умножаю на вектор 
        matrix = np.eye(N0) @ matrix - 2 * u.transpose() @ matrix_1 # и тут снова идет умножение вектора на матрицу
        # Вроде, у меня получилось написать так, чтобы было умножение вектора на матрицу, а потом снова умножение
        # вектора на матрицу, а не чтобы было умножение вектора на вектор, а потом умножение матрицы на матрицы
        matrix_u[i] = u[0] # это просто заполняем матрицу нашими векторами u 
        # матрица matrix_u устроена так, что i - ая строчка соответствует i-ому вектора
    r = matrix
    return r,matrix_u

In [600]:
def qr2(a,matrix_u): #создадим теперь функцию, которая получает матрицу a и набор векторов u
    N0 = matrix_u.shape[0] #кол-во строк
    N1 = matrix_u.shape[1] #кол-во столбцов
    matrix = a.copy()
    for i in range(0,N0):#N0 - это кол-во строк, то есть это кол-во преобразований, которые произошли
                         # значит нам нужно произвести N0 произведений
        u = np.zeros((1,N1),dtype = float)
        u[0] = matrix_u[i] # вытаскиваем i - ый вектор из набора(I'm so sorry, нормально делать это так и не научился)
        matrix_1 = u @ matrix # заставляем также сначала умножем вектор на матрицу(если это, конечно, так работает)
                              #чтобы получить выигрыш в сложности
        matrix = np.eye(N1) @ matrix - 2 * u.transpose() @ matrix_1 # и производим окончательное преобразование зацикленное
    r = matrix
    return r

Теперь мы можем с помощью qr11 получить r и набор векторов, сделаем небольшую проверку

In [628]:
matrix_u = qr11(a)[1]
rr1 = qr11(a)[0]

In [629]:
qr2(a , matrix_u)

array([[ 1.40152769,  1.2514379 ,  0.89523615],
       [ 0.        ,  0.84158252,  0.68447942],
       [ 0.        , -0.        , -0.13872729],
       [ 0.        ,  0.        ,  0.31721312],
       [ 0.        , -0.        , -0.42684399]])

In [631]:
rr1

array([[ 1.40152769,  1.2514379 ,  0.89523615],
       [ 0.        ,  0.84158252,  0.68447942],
       [ 0.        , -0.        , -0.13872729],
       [ 0.        ,  0.        ,  0.31721312],
       [ 0.        , -0.        , -0.42684399]])

In [632]:
print(rr1 - qr2(a, matrix_u))

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


Все получилось. И у этих двух функций совпадают R